# Red Neuronal Convolucional
## Parte 1: Construyendo la RNC
### Importar librerias y paquetes de keras

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


### Inicializando la RNC

In [2]:
classifier = Sequential()

### Paso 1 - Convolución

In [3]:
classifier.add(Convolution2D(
    32,                         # 32 filtros
    (3, 3),                     # filtros de 3x3
    input_shape = (64, 64, 3),  # entrada imagenes rgb 64x64
    activation = 'relu'         # no-lineal
))

### Paso 2 - Pooling

In [4]:
classifier.add(MaxPooling2D(
    pool_size = (2,2) # pooling de 2x2
    # stride por default es 2 (pool_size)
))

### EXTRA: Segunda Convolución
Tras ver los resultados con una sola convolución, se decidió añadir una segunda convolución para mejorar los resultados
- Resultados con 1 convolución   : loss: 0.5028 - acc: 0.7495 - val_loss: 0.5097 - val_acc: 0.7680
- Resultados con 2 convoluciónes : loss: 0.2288 - acc: 0.9071 - val_loss: 0.5439 - val_acc: 0.8105

In [5]:
classifier.add(Convolution2D(
    32,                         # 32 filtros
    (3, 3),                     # filtros de 3x3
    activation = 'relu'         # no-lineal
))
classifier.add(MaxPooling2D(
    pool_size = (2,2) # pooling de 2x2
    # stride por default es 2 (pool_size)
))

### Paso 3 - Flatting

In [6]:
classifier.add(Flatten()) # keras sabe que pedo

### Paso 4 - Conexión completa

In [7]:
# Capa oculta
classifier.add(Dense(
    units = 128,         # numero de neuronas
    activation = "relu"
))
# Capa de salida
classifier.add(Dense(
    units = 1,             # perro o gato
    activation = 'sigmoid'
))

### Compilando la RNC

In [8]:
classifier.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

## Parte 2: Alimentando la RNC con imagenes

In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255,       # valores entre 0 y 1
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set', # ubicacion de imagenes
    target_size = (64, 64), # tamaño de imagenes
    batch_size = 32,
    class_mode = 'binary'
)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)
classifier.fit_generator(
    training_set,
    steps_per_epoch = 250,       # train_size / batch_size
    epochs = 25,
    validation_data = test_set,
    validation_steps = 62,       # test_size / batch_size
    workers = 4                  # número de procesos
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 71s 282ms/step - loss: 0.6406 - acc: 0.6430 - val_loss: 0.6237 - val_acc: 0.6585
Epoch 2/25
250/250 [==============================] - 70s 282ms/step - loss: 0.5957 - acc: 0.6824 - val_loss: 0.6031 - val_acc: 0.6725
Epoch 3/25
250/250 [==============================] - 72s 287ms/step - loss: 0.5544 - acc: 0.7140 - val_loss: 0.5410 - val_acc: 0.7520
Epoch 4/25
250/250 [==============================] - 72s 286ms/step - loss: 0.5212 - acc: 0.7356 - val_loss: 0.5148 - val_acc: 0.7615
Epoch 5/25
250/250 [==============================] - 70s 278ms/step - loss: 0.4952 - acc: 0.7549 - val_loss: 0.4971 - val_acc: 0.7740
Epoch 6/25
250/250 [==============================] - 65s 260ms/step - loss: 0.4713 - acc: 0.7738 - val_loss: 0.4948 - val_acc: 0.7710
Epoch 7/25
250/250 [==============================] - 69s 276ms/step - loss: 0.4657 - acc: 0.7732 - val_lo

### Predicción

In [27]:
import numpy as np
from keras.preprocessing import image

# load image with size 64x64
test_image = image.load_img('dataset/single_prediction/Ragdoll_202.jpg', target_size = (64, 64))
# add color chanel dimension
test_image = image.img_to_array(test_image)
# add another dimension for the batch
test_image = np.expand_dims(test_image, 0)
result = classifier.predict(test_image)
print(result[0][0])

1.0
